In [1]:
%pylab nbagg
from tvb.simulator.lab import *
import os.path
from matplotlib import colors, cm
import time
import scipy.signal as sig

import numpy as np
import pandas as pd
import scipy.io

# downloaded library for peak detection in z time series
import peakdetect
from runmainsim import *
import tvbsim

%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
def renamefiles(patient, project_dir):
    ####### Initialize files needed to 
    # convert seeg.xyz to seeg.txt file
    sensorsfile = os.path.join(project_dir, "seeg.xyz")
    newsensorsfile = os.path.join(project_dir, "seeg.txt")
    try:
        os.rename(sensorsfile, newsensorsfile)
    except:
        print "Already renamed seeg.xyz possibly!"

    # convert gain_inv-square.mat file into gain_inv-square.txt file
    gainmatfile = os.path.join(project_dir, "gain_inv-square.mat")
    newgainmatfile = os.path.join(project_dir, "gain_inv-square.txt")
    try:
        os.rename(gainmatfile, newgainmatfile)
    except:
        print "Already renamed gain_inv-square.mat possibly!" 

# 1. Set Up Simulation Configuration First - Move Contact
Here, set everything from the ez region, connectivity, node model, coupling, monitors, integrator, and parameters.

In [25]:
patients = ['id001_ac', 'id002_cj', 'id014_rb']
patients = ['id014_rb']
patients = ['id001_ac']
for patient in patients:
    print patient
#     patient='cj'
    # 1000 = 1 second
    samplerate = 1000 # Hz
    sim_length = 240*samplerate

    ezregion = ['ctx-lh-bankssts', 'ctx-lh-cuneus']
    ezregion = ['ctx-lh-bankssts']
#     ezregion = []
    pzregion = ['ctx-lh-cuneus']
    pzregion = []

    outputdir = os.path.join('/Users/adam2392/Documents/pydata/tvb_forwardsim/', patient)
    if not os.path.exists(outputdir):
        os.mkdir(outputdir)
    filename = os.path.join(outputdir, patient+'_nonmovedsim_nez'+str(len(ezregion))+\
                                        '_npz'+str(len(pzregion))+'.npz')

    period = 1
    # x0c value = -2.05
    x0norm=-2.5
    x0ez=-1.6
    x0pz=-2.04
    
    # intialized hard coded parameters
    epileptor_r = 0.0002       # Temporal scaling in the third state variable
    epiks = -0.5                 # Permittivity coupling, fast to slow time scale
    epitt = 0.05               # time scale of simulation
    epitau = 10                # Temporal scaling coefficient in fifth st var

    # depends on epileptor variables of interest: it is where the x2-y2 var is
    varindex = [1]
    
    # parameters for heun-stochastic integrator
    heun_ts = 0.05
    noise_cov = np.array([0.001, 0.001, 0.,\
                    0.0001, 0.0001, 0.])
    
    root_dir = os.getcwd()
    project_dir = os.path.join(root_dir, "metadata/"+patient)
    print "Project directory for meta data is : ", project_dir
    print "File to be saved is: ", filename
    renamefiles(patient, project_dir)
    
    ####### Initialize files needed to 
    # convert seeg.xyz to seeg.txt file
    sensorsfile = os.path.join(project_dir, "seeg.txt")
    # convert gain_inv-square.mat file into gain_inv-square.txt file
    gainmatfile = os.path.join(project_dir, "gain_inv-square.txt")
    
    confile = os.path.join(project_dir, "connectivity.zip")
    ####################### 1. Structural Connectivity ########################
    con = initconn(confile)
    
    # extract the seeg_xyz coords and the region centers
    seeg_xyz = tvbsim.util.extractseegxyz(sensorsfile)
    seeg_labels = seeg_xyz.index.values
    region_centers = con.centres
    regions = con.region_labels
    num_regions = len(regions)
    
    # initialize object to assist in moving seeg contacts
    movecontact = tvbsim.util.MoveContacts(seeg_labels, seeg_xyz, 
                                       regions, region_centers, True)
    
    ezindices = movecontact.getindexofregion(ezregion)
    pzindices = movecontact.getindexofregion(pzregion)
    
    # hardcode 1 ez index to get moved contacts
    ezindex = ezindices[0]
    
    # find the closest contact index and distance
    seeg_index, distance = movecontact.findclosestcontact(ezindex)
#     try:
    # get the modified seeg xyz and gain matrix
    modseeg = movecontact.movecontact(ezindex, seeg_index)
    modgain = movecontact.simplest_gain_matrix(modseeg)

    # modify the config of the monitors
    monitors[1].sensors.locations = modseeg
    monitors[1].gain = modgain
#     except:
#         print "not modifying seeg xyz and gain matrix"
    
    ####################### 2. Neural Mass Model @ Nodes ######################
    epileptors = initepileptor(epileptor_r, epiks, epitt, epitau, x0norm, \
                              x0ez, x0pz, ezindices, pzindices, num_regions)    
    ####################### 3. Integrator for Models ##########################
    heunint = initintegrator(heun_ts, noise_cov)
    ################## 4. Difference Coupling Between Nodes ###################
    coupl = initcoupling(a=1.)
    ############## 5. Import Sensor XYZ, Gain Matrix For Monitors #############
    monitors = initmonitors(period, sensorsfile, gainmatfile, varindex)
    
    # get initial conditions and then setup entire simulation configuration
    initcond = initconditions(x0norm, num_regions)
#     sim, configs = setupconfig(epileptors, con, coupl, heunint, monitors, initcond)

id001_ac
Project directory for meta data is :  /Users/adam2392/Documents/tvb/metadata/id001_ac
File to be saved is:  /Users/adam2392/Documents/pydata/tvb_forwardsim/id001_ac/id001_ac_nonmovedsim_nez1_npz0.npz
Already renamed seeg.xyz possibly!
Already renamed gain_inv-square.mat possibly!
OR'1
OR
[-17.97  -4.23   4.62]


 movecontact function summary: 

Closest contact to ezregion:  [-46.4062 -68.0003  14.9149]  is  OR'1
That is located at:  [-17.97  -4.23   4.62]
It will move:  [-28.436199999999999, -63.770299999999992, 10.294899999999998]
New location after movement is [-46.4062 -68.0003  14.9149]





In [5]:
times, epilepts, seegts = runsim(sim, sim_length)

postprocessor = PostProcess(epilepts, seegts, times)
######################## POST PROCESSING #################################
# post process by cutting off first 5 seconds of simulation
# for now, don't, since intiial conditions
times, epits, seegts, zts = PostProcess.postprocts(samplerate)

# get the onset, offset times
onsettimes = None
offsettimes = None
try:
    onsettimes, offsettimes = PostProcess.findonsetoffset(zts[ezindices, :].squeeze())
except:
    print "Still not working..."

######################## SAVING ALL DATA #################################
regions = configs.connectivity.region_labels
# Save files
meta = {
    'x0ez':x0ez,
    'x0pz':x0pz,
    'x0norm':x0norm,
    'regions': regions,
    'regions_centers': configs.connectivity.centres,
    'seeg_contacts': configs.monitors[1].sensors.labels,
    'seeg_xyz': configs.monitors[1].sensors.locations,
    'ez': regions[ezindices],
    'pz': regions[pzindices],
    'ezindices': ezindices,
    'pzindices': pzindices,
    'onsettimes':onsettimes,
    'offsettimes':offsettimes,
    'patient':patient,
}

# save tseries
np.savez(filename, epits=epits, seegts=seegts, \
         times=times, zts=zts, metadata=meta)

# 2. Apply EZ To Closest Region

Here, set everything from the ez region, connectivity, node model, coupling, monitors, integrator, and parameters.

In [34]:
for idx, label in enumerate(seeg_labels):
    region_index, distance = movecontact.getregionsforcontacts(label)
    
    if idx == 0:
        mindist = distance
        minregion = region_index
        mincontact = label
    else:
        if distance < mindist:
            mindist = distance
            minregion = region_index
            mincontact = label

print mincontact
print mindist
print minregion
print regions[minregion]
# sim, configs = setupconfig(epileptors, con, coupl, heunint, monitors, initcond)

PM2
[ 1.83309017]
[75]
['ctx-rh-superiorfrontal']


In [40]:
patient = 'id001_ac'
# 1000 = 1 second
samplerate = 1000 # Hz
sim_length = 240*samplerate

ezregion = ['ctx-lh-bankssts', 'ctx-lh-cuneus']
ezregion = ['ctx-lh-bankssts']
ezregion = regions[minregion]
#     ezregion = []
pzregion = ['ctx-lh-cuneus']
pzregion = []

outputdir = os.path.join('/Users/adam2392/Documents/pydata/tvb_forwardsim/', patient)
if not os.path.exists(outputdir):
    os.mkdir(outputdir)
filename = os.path.join(outputdir, patient+'closeregion_sim_nez'+str(len(ezregion))+\
                                    '_npz'+str(len(pzregion))+'.npz')

period = 1
# x0c value = -2.05
x0norm=-2.5
x0ez=-1.6
x0pz=-2.04

# intialized hard coded parameters
epileptor_r = 0.0002       # Temporal scaling in the third state variable
epiks = -0.5                 # Permittivity coupling, fast to slow time scale
epitt = 0.05               # time scale of simulation
epitau = 10                # Temporal scaling coefficient in fifth st var

# depends on epileptor variables of interest: it is where the x2-y2 var is
varindex = [1]

# parameters for heun-stochastic integrator
heun_ts = 0.05
noise_cov = np.array([0.001, 0.001, 0.,\
                0.0001, 0.0001, 0.])

print "output dir is: ", outputdir
print filename

output dir is:  /Users/adam2392/Documents/pydata/tvb_forwardsim/id001_ac
/Users/adam2392/Documents/pydata/tvb_forwardsim/id001_ac/id001_accloseregion_sim_nez1_npz0.npz


In [41]:
root_dir = os.getcwd()
project_dir = os.path.join(root_dir, "metadata/"+patient)
print "Project directory for meta data is : ", project_dir
print "File to be saved is: ", filename
renamefiles(patient, project_dir)

####### Initialize files needed to 
# convert seeg.xyz to seeg.txt file
sensorsfile = os.path.join(project_dir, "seeg.txt")
# convert gain_inv-square.mat file into gain_inv-square.txt file
gainmatfile = os.path.join(project_dir, "gain_inv-square.txt")

confile = os.path.join(project_dir, "connectivity.zip")

Project directory for meta data is :  /Users/adam2392/Documents/tvb/metadata/id001_ac
File to be saved is:  /Users/adam2392/Documents/pydata/tvb_forwardsim/id001_ac/id001_accloseregion_sim_nez1_npz0.npz
Already renamed seeg.xyz possibly!
Already renamed gain_inv-square.mat possibly!


In [43]:
####################### 1. Structural Connectivity ########################
con = initconn(confile)

# extract the seeg_xyz coords and the region centers
seeg_xyz = tvbsim.util.extractseegxyz(sensorsfile)
seeg_labels = seeg_xyz.index.values
region_centers = con.centres
regions = con.region_labels
num_regions = len(regions)

# initialize object to assist in moving seeg contacts
movecontact = tvbsim.util.MoveContacts(seeg_labels, seeg_xyz, 
                                   regions, region_centers, True)

ezindices = movecontact.getindexofregion(ezregion)
pzindices = movecontact.getindexofregion(pzregion)

####################### 2. Neural Mass Model @ Nodes ######################
epileptors = initepileptor(epileptor_r, epiks, epitt, epitau, x0norm, \
                          x0ez, x0pz, ezindices, pzindices, num_regions)    
####################### 3. Integrator for Models ##########################
heunint = initintegrator(heun_ts, noise_cov)
################## 4. Difference Coupling Between Nodes ###################
coupl = initcoupling(a=1.)
############## 5. Import Sensor XYZ, Gain Matrix For Monitors #############
monitors = initmonitors(period, sensorsfile, gainmatfile, varindex)

# get initial conditions and then setup entire simulation configuration
initcond = initconditions(x0norm, num_regions)

In [44]:
sim, configs = setupconfig(epileptors, con, coupl, heunint, monitors, initcond)

In [ ]:
times, epilepts, seegts = runsim(sim, sim_length)

postprocessor = PostProcess(epilepts, seegts, times)
######################## POST PROCESSING #################################
# post process by cutting off first 5 seconds of simulation
# for now, don't, since intiial conditions
times, epits, seegts, zts = PostProcess.postprocts(samplerate)

# get the onset, offset times
onsettimes = None
offsettimes = None
try:
    onsettimes, offsettimes = PostProcess.findonsetoffset(zts[ezindices, :].squeeze())
except:
    print "Still not working..."

######################## SAVING ALL DATA #################################
regions = configs.connectivity.region_labels
# Save files
meta = {
    'x0ez':x0ez,
    'x0pz':x0pz,
    'x0norm':x0norm,
    'regions': regions,
    'regions_centers': configs.connectivity.centres,
    'seeg_contacts': configs.monitors[1].sensors.labels,
    'seeg_xyz': configs.monitors[1].sensors.locations,
    'ez': regions[ezindices],
    'pz': regions[pzindices],
    'ezindices': ezindices,
    'pzindices': pzindices,
    'onsettimes':onsettimes,
    'offsettimes':offsettimes,
    'patient':patient,
}

# save tseries
np.savez(filename, epits=epits, seegts=seegts, \
         times=times, zts=zts, metadata=meta)